In [7]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import ssl

ssl._create_default_https_context = ssl._create_unverified_context


# def load_cifar10_dataset(folder_path):
#     train_data = []
#     train_labels = []

#     for i in range(1, 6):
#         with open(f"{folder_path}/data_batch_{i}", 'rb') as f:
#             data = pickle.load(f, encoding='bytes')
#             train_data.append(data[b'data'])
#             train_labels.extend(data[b'labels'])

#     with open(f"{folder_path}/test_batch", 'rb') as f:
#         test_data = pickle.load(f, encoding='bytes')
#         test_images = test_data[b'data']
#         test_labels = test_data[b'labels']

#     train_images = np.vstack(train_data).reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    
#     return train_images, np.array(train_labels), test_images.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1), np.array(test_labels)

# # Specify the folder path where your CIFAR-10 dataset is located
# cifar10_folder = "./imageProcessing/Week_10/cifar-10-batches-py"

# # # Load the dataset
# train_images, train_labels, test_images, test_labels = load_cifar10_dataset(cifar10_folder)
# Now you can use train_images, train_labels, test_images, and test_labels for your machine learning tasks.


In [8]:
data = tf.keras.datasets.cifar10.load_data()
(train_images, train_labels),(test_images, test_labels) = data
train_images, test_images = train_images / 255.0, test_images / 255.0

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

def residual_block(x, filters, kernel_size=3, stride=1):
    y = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    y = BatchNormalization()(y)
    y = ReLU()(y)
    y = Conv2D(filters, kernel_size=kernel_size, padding='same')(y)
    y = BatchNormalization()(y)
    
    if stride > 1:
        x = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(x)
    
    out = Add()([x, y])
    out = ReLU()(out)
    return out

def resnet(num_blocks=3):
    input_layer = Input(shape=(32, 32, 3))
    x = Conv2D(64, 3, strides=1, padding='same')(input_layer)  # Changed kernel size to 3x3
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = AveragePooling2D(3, strides=2, padding='same')(x)
    
    for _ in range(num_blocks):
        x = residual_block(x, 64)
    
    x = AveragePooling2D(8)(x)
    x = Flatten()(x)
    x = Dense(10, activation='softmax')(x)
    
    return Model(inputs=input_layer, outputs=x)
     
# You can increase num_blocks for a deeper network
model = resnet(num_blocks=3)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Define callbacks (optional but useful)
checkpoint = ModelCheckpoint("resnet_cifar10.h5", monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, verbose=1, restore_best_weights=True)

# Train the model
history = model.fit(train_images, train_labels, 
                    epochs=50, 
                    batch_size=32, 
                    validation_split=0.2, 
                    verbose = 1,
                    callbacks=[checkpoint, early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/50
1250/1250 [==============================] - ETA: 0s - loss: 1.3187 - accuracy: 0.5253
Epoch 1: val_accuracy improved from -inf to 0.56780, saving model to resnet_cifar10.h5
1250/1250 [==============================] - 163s 130ms/step - loss: 1.3187 - accuracy: 0.5253 - val_loss: 1.2777 - val_accuracy: 0.5678
Epoch 2/50
1250/1250 [==============================] - ETA: 0s - loss: 0.9290 - accuracy: 0.6765
Epoch 2: val_accuracy improved from 0.56780 to 0.60140, saving model to resnet_cifar10.h5
1250/1250 [==============================] - 177s 141ms/step - loss: 0.9290 - accuracy: 0.6765 - val_loss: 1.0983 - val_accuracy: 0.6014
Epoch 3/50
1250/1250 [==============================] - ETA: 0s - loss: 0.7873 - accuracy: 0.7245
Epoch 3: val_accuracy did not improve from 0.60140
1250/1250 [==============================] - 169s 135ms/step - loss: 0.7873 - accuracy: 0.7245 - val_loss: 1.6769 - val_accuracy: 0.4810
Epoch 4/50
1250/1250 [==============================] - ETA: 0s - l

In [14]:

# Train the model
history = model.fit(train_images, train_labels, 
                    epochs=50, 
                    batch_size=128, 
                    validation_split=0.2, 
                    verbose = 1,
                    callbacks=[checkpoint, early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/50
313/313 [==============================] - ETA: 0s - loss: 0.0745 - accuracy: 0.9795
Epoch 1: val_accuracy improved from 0.80630 to 0.84430, saving model to resnet_cifar10.h5
313/313 [==============================] - 156s 499ms/step - loss: 0.0745 - accuracy: 0.9795 - val_loss: 0.5349 - val_accuracy: 0.8443
Epoch 2/50
313/313 [==============================] - ETA: 0s - loss: 0.0451 - accuracy: 0.9923
Epoch 2: val_accuracy did not improve from 0.84430
313/313 [==============================] - 153s 490ms/step - loss: 0.0451 - accuracy: 0.9923 - val_loss: 0.5672 - val_accuracy: 0.8428
Epoch 3/50
313/313 [==============================] - ETA: 0s - loss: 0.0347 - accuracy: 0.9954
Epoch 3: val_accuracy did not improve from 0.84430
313/313 [==============================] - 159s 509ms/step - loss: 0.0347 - accuracy: 0.9954 - val_loss: 0.5852 - val_accuracy: 0.8392
Epoch 4/50
313/313 [==============================] - ETA: 0s - loss: 0.0276 - accuracy: 0.9972
Epoch 4: val_accura

In [15]:
history = model.fit(train_images, train_labels, 
                    epochs=15, 
                    batch_size=256, 
                    validation_split=0.2, 
                    verbose = 1,
                    callbacks=[checkpoint, early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/15
157/157 [==============================] - ETA: 0s - loss: 0.0464 - accuracy: 0.9916
Epoch 1: val_accuracy did not improve from 0.84430
157/157 [==============================] - 158s 1s/step - loss: 0.0464 - accuracy: 0.9916 - val_loss: 0.6233 - val_accuracy: 0.8314
Epoch 2/15
157/157 [==============================] - ETA: 0s - loss: 0.0325 - accuracy: 0.9963
Epoch 2: val_accuracy did not improve from 0.84430
157/157 [==============================] - 154s 979ms/step - loss: 0.0325 - accuracy: 0.9963 - val_loss: 0.5813 - val_accuracy: 0.8383
Epoch 3/15
157/157 [==============================] - ETA: 0s - loss: 0.0248 - accuracy: 0.9981
Epoch 3: val_accuracy did not improve from 0.84430
157/157 [==============================] - 153s 974ms/step - loss: 0.0248 - accuracy: 0.9981 - val_loss: 0.5916 - val_accuracy: 0.8398
Epoch 4/15
157/157 [==============================] - ETA: 0s - loss: 0.0187 - accuracy: 0.9988
Epoch 4: val_accuracy did not improve from 0.84430
157/157 [=

In [17]:
history = model.fit(train_images, train_labels, 
                    epochs=15, 
                    batch_size=512, 
                    validation_split=0.2, 
                    verbose = 1,
                    callbacks=[checkpoint, early_stopping])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/15
79/79 [==============================] - ETA: 0s - loss: 0.5634 - accuracy: 0.8104
Epoch 1: val_accuracy did not improve from 0.84510
79/79 [==============================] - 164s 2s/step - loss: 0.5634 - accuracy: 0.8104 - val_loss: 1.0353 - val_accuracy: 0.6867
Epoch 2/15
79/79 [==============================] - ETA: 0s - loss: 0.3691 - accuracy: 0.8711
Epoch 2: val_accuracy did not improve from 0.84510
79/79 [==============================] - 161s 2s/step - loss: 0.3691 - accuracy: 0.8711 - val_loss: 0.7974 - val_accuracy: 0.7311
Epoch 3/15
79/79 [==============================] - ETA: 0s - loss: 0.2882 - accuracy: 0.8989
Epoch 3: val_accuracy did not improve from 0.84510
79/79 [==============================] - 164s 2s/step - loss: 0.2882 - accuracy: 0.8989 - val_loss: 0.6472 - val_accuracy: 0.7834
Epoch 4/15
79/79 [==============================] - ETA: 0s - loss: 0.2308 - accuracy: 0.9195
Epoch 4: val_accuracy did not improve from 0.84510
79/79 [=======================